In [3]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
# list with input parameters
a = [[0.0009,0.0011,0.0013,0.0015,0.0017,0.0019,0.0021,0.0023,0.0025,0.0027],
     [2.3,2.7,3.1,3.5,3.9,4.3,4.7,5.1,5.5,5.9],
     [7.3,7.6,7.9,8.2,8.5,8.8,9.1,9.4,9.7,10],
     [0.77,0.83,0.89,0.95,1.01,1.07,1.13,1.19,1.25,1.31],
     [0.011,0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02]]
L=pd.DataFrame(itertools.product(*a))

In [4]:
n=100
for i in range(len(L)):
    indice=i
    v=L[0][i]
    alpha=L[1][i]
    l=L[2][i]
    F0=L[3][i]
    sigma=L[4][i]

    def phi(w):
      return (F0-sigma)/(1.0 + w/(1.0-sigma))

    num_moving_blocks=0
    def movement(r,t,movingpoints,tpoints):

        num_moving_blocks = 0
        omega_i = r[n:]

        for i in range(0,n):
          if (omega_i[i] > -v  ):
            num_moving_blocks += 1

        movementpoints.append(num_moving_blocks)
        return num_moving_blocks

    def f(r,t):

        U_i=r[:n]
        omega_i=r[n:]

        fU_i = np.empty(n,float)
        fomega_i = np.empty(n,float)

        fU_i[0] = omega_i[0]
        s0 = (l**2)*(U_i[1]-U_i[0]) - U_i[0]
        if (omega_i[0] <= -v and s0 <= F0):
          omega_i[0] = -v
          fomega_i[0] = 0
          #U_i[0]=-v*h
        else:
          fomega_i[0] = s0 - phi(2*alpha*v + 2*alpha*omega_i[0])

        for j in range(1,n-1):
          fU_i[j] = omega_i[j]
          s = (l**2)*(U_i[j+1]-2*U_i[j]+U_i[j-1]) - U_i[j]
          if (omega_i[j] <= -v and s <= F0):
            omega_i[j] = -v
            fomega_i[j] = 0
            #U_i[j]=-v*h*tpoints[j]
          else:
            fomega_i[j] = s - phi(2*alpha*v + 2*alpha*omega_i[j])

        fU_i[n-1] = omega_i[n-1]
        sn = -(l**2)*(U_i[n-1]-U_i[n-2]) - U_i[n-1]
        if (omega_i[n-1] <= -v and  sn <= F0) :
          omega_i[n-1] = -v
          fomega_i[n-1] = 0
          #U_i[n-1]=-v*h*tpoints[n-1]
        else:
          fomega_i[n-1] = sn - phi(2*alpha*v + 2*alpha*omega_i[n-1])

        return np.concatenate([fU_i,fomega_i])

    h = 0.01
    tpoints = []
    movementpoints = []

    PV = [] #Crea una lista de listas donde se almacenarán los valores de las posiciones y velocidades.
    for i in range(2*n):
        PV.append([])

    vi=[]
    for i in range(n):
      vi.append(-v)

    pi=np.array([2.07826932, 2.07323611, 2.07332616, 2.10142405, 2.0678997 ,
       2.0380718 , 2.02644764, 1.99705727, 1.95817644, 1.93869478,
       1.93349957, 1.93982916, 1.94022151, 1.95331413, 1.98005875,
       2.01064141, 2.06581628, 2.07971485, 2.11202169, 2.17213122,
       2.2209527 , 2.2724812 , 2.2859322 , 2.23506666, 2.18551569,
       2.13882499, 2.04194555, 1.96887861, 1.9243071 , 1.87314174,
       1.84878959, 1.82046868, 1.80606972, 1.81729648, 1.82274989,
       1.84034566, 1.87317978, 1.87509397, 1.90265871, 1.87268564,
       1.87098663, 1.89382655, 1.90068585, 1.92306531, 1.94183919,
       1.92335701, 1.90342386, 1.90845925, 1.86961215, 1.83946077,
       1.81765079, 1.79458261, 1.78451695, 1.77741826, 1.77528937,
       1.74218947, 1.7140009 , 1.69240053, 1.6725756 , 1.65961241,
       1.65120093, 1.64516666, 1.6497147 , 1.66447789, 1.7023125 ,
       1.74375292, 1.73342154, 1.74385603, 1.73680559, 1.72701017,
       1.72000761, 1.73725502, 1.76532645, 1.81430041, 1.82007717,
       1.80258275, 1.77859816, 1.77240313, 1.77604231, 1.77958612,
       1.80274729, 1.84480897, 1.87548607, 1.91505901, 1.94918904,
       1.96182367, 1.98778176, 2.03128942, 2.05030746, 2.01227454,
       1.96814153, 1.92448248, 1.88716587, 1.84102669, 1.81958678,
       1.80061531, 1.79937923, 1.80211039, 1.80563773, 1.79793826])
    r=np.concatenate([pi,vi])


    c=-1
    aux=c
    auxblocks=[]
    for k in range(1,2000):

      aux1=aux+1
      i=c+1
      tiempoaux=[]

      while num_moving_blocks <= n:
        t = i*h
        for j in range(2*n):
          PV[j].append(r[j])


        k1 = h*f(r,t)
        k2 = h*f(r+0.5*k1, t+0.5*h)
        k3 = h*f(r+0.5*k2, t+0.5*h)
        k4 = h*f(r+k3, t+h)
        r += (k1+2*k2+2*k3+k4)/6

        tpoints.append(t)
        num_moving_blocks = movement(r,t,movementpoints,tpoints)

        if (num_moving_blocks >= 1 or t > 20000):
          break
        i+=1

      c=i+1
      while num_moving_blocks >= 0:
        t = c*h
        for j in range(2*n):
          PV[j].append(r[j])


        k1 = h*f(r,t)
        k2 = h*f(r+0.5*k1, t+0.5*h)
        k3 = h*f(r+0.5*k2, t+0.5*h)
        k4 = h*f(r+k3, t+h)
        r += (k1+2*k2+2*k3+k4)/6

        tpoints.append(t)
        tiempoaux.append(t)
        num_moving_blocks = movement(r,t,movementpoints,tpoints)
        if (num_moving_blocks >= 50 or tiempoaux[-1]-tiempoaux[0] > 100 or t > 20000) :
          auxblocks.append(num_moving_blocks)
        if (num_moving_blocks < 1 or tiempoaux[-1]-tiempoaux[0] > 100 or t > 20000):
          break
        c+=1

      aux=c
      D=[]
      for i in range(n):
        Di=[]
        for j in range(aux1,len(tpoints)):
          Di.append(PV[i][j]+v*tpoints[j])
        D.append(Di)

      M=[]
      for j in range(len(tpoints)-aux1):
          suma=sum([fila[j] for fila in D])
          M.append(suma)

      mag0=M[-1]-M[0]
      #print(k,',',num_moving_blocks,',',mag0,',',t)
      if (len(auxblocks)>=1 or t > 20000):
        break
      k


    print(indice,',',v,',',alpha,',',l,',',F0,',',sigma,',',k,',',mag0,',',tiempoaux[0],',',tiempoaux[-1],',',tiempoaux[-1]-tiempoaux[0])

KeyboardInterrupt: 